In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
import pickle


parkinsons_dataset = pd.read_csv("../datasets/parkinsons.csv")


x = parkinsons_dataset.drop(columns=['name', 'status'], axis=1)
y = parkinsons_dataset['status']


scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)


x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)


model = SVC(random_state=42)


param_grid = {
    'model__C': [0.1, 1, 10, 100],
    'model__gamma': [1, 0.1, 0.01, 0.001],
    'model__kernel': ['rbf', 'linear']
}


pipeline = Pipeline([
    ('oversample', SMOTE(random_state=42)),
    ('undersample', RandomUnderSampler(random_state=42)),
    ('model', model)
])


grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(x_train, y_train)


best_model = grid_search.best_estimator_


cv_scores = cross_val_score(best_model, x_train, y_train, cv=5)
print(f"SVM Cross-Validation Accuracy: {np.mean(cv_scores) * 100:.2f}%")


best_model.fit(x_train, y_train)


train_predictions = best_model.predict(x_train)
train_accuracy = accuracy_score(y_train, train_predictions)


test_predictions = best_model.predict(x_test)
test_accuracy = accuracy_score(y_test, test_predictions)


print(f"SVM Training Accuracy: {train_accuracy * 100:.2f}%")
print(f"SVM Test Accuracy: {test_accuracy * 100:.2f}%")


filename = 'parkinsons_model.sav'
pickle.dump(best_model, open(filename, 'wb'))

Fitting 5 folds for each of 32 candidates, totalling 160 fits
SVM Cross-Validation Accuracy: 92.30%
SVM Training Accuracy: 99.36%
SVM Test Accuracy: 94.87%
